<a href="https://colab.research.google.com/github/alfinmaulani/skripsi/blob/master/w2v(semua).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import io
import os
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
import gensim

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
url0='https://raw.githubusercontent.com/alfinmaulani/skripsi/master/dataISEAR.csv'
# url1 = 'https://raw.githubusercontent.com/alfinmaulani/skripsi/master/dataisear(preputama).csv'
# url2 = 'tba'
df = pd.read_csv(url0)
# df.iloc[:,1]

In [3]:
df.emosi.value_counts()

joy        1092
sadness    1082
anger      1079
fear       1076
shame      1071
disgust    1066
guilt      1050
Name: emosi, dtype: int64

In [4]:
df.head()

,emosi,dokumen
0,joy,On days when I feel close to my partner and ot...
1,fear,Every time I imagine that someone I love or I ...
2,anger,When I had been obviously unjustly treated and...
3,sadness,When I think about the short time that we live...
4,disgust,At a gathering I found myself involuntarily si...


lower case

In [6]:
df['dokumen']=df['dokumen'].str.lower()
# print(df.iloc[4,1])

Pembersihan noise (over enter and space)

In [7]:
df=df.replace(' \n', ' ', regex=True)
df=df.replace('   ', ' ', regex=True)
df=df.replace('  ', ' ', regex=True)

expand contractions

In [8]:
!pip install contractions
import contractions

     |████████████████████████████████| 284 kB 8.4 MB/s 
     |████████████████████████████████| 321 kB 47.7 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85453 sha256=3a513b598826af8fd771474d69daf7cb9c2a42a7ac0d3edb7354bf22bf9c0071
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [9]:
df[df['dokumen'].str.contains("couldn't")]

,emosi,dokumen
124,disgust,"when i saw a picture, in the gothenburg post, ..."
526,anger,a friend suggested in a round-about way that t...
545,fear,"i was rock-climbing and absailing & felt ok, b..."
553,anger,"getting a terrible exam mark, which i couldn't..."
673,fear,"after seeing a terror movie at the cinema, i w..."
...,...,...
6589,joy,when my boyfriend came form rumphi to lilongwe...
6592,sadness,i wrote a letter to my boyfriend but he couldn...
6595,guilt,one day i found an incontinent man in the ward...
6624,joy,it was in 1983 september when mce results were...


In [10]:
expanded_words = []    

In [11]:
for word in df.dokumen:
  expanded_words.append(contractions.fix(word))

In [12]:
np.shape(expanded_words)

(7516,)

In [13]:
df.dokumen=expanded_words

In [14]:
df[df['dokumen'].str.contains("couldn't")]

,emosi,dokumen


In [15]:
df.dokumen[124]

'when i saw a picture, in the gothenburg post, of a totally demolished car with two fatally injured passengers. the car was so badly damaged that they could not get the bodies out at the scene of the accident.'

cleansing

In [16]:
df['dokumen'] = df['dokumen'].str.replace(r'[^\w\s]+', '')

In [17]:
df.shape

(7516, 2)

remove stopword

In [18]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
pat = r'\b(?:{})\b'.format('|'.join(stop))
df['dokumen'] = df['dokumen'].str.replace(pat, '')
df['dokumen'] = df['dokumen'].str.replace(r'\s+', ' ')

In [20]:
df.head()

,emosi,dokumen
0,joy,days feel close partner friends feel peace al...
1,fear,every time imagine someone love could contact ...
2,anger,obviously unjustly treated possibility elucid...
3,sadness,think short time live relate periods life thi...
4,disgust,gathering found involuntarily sitting next tw...


In [21]:
df.dokumen

0        days feel close partner friends feel peace al...
1       every time imagine someone love could contact ...
2        obviously unjustly treated possibility elucid...
3        think short time live relate periods life thi...
4        gathering found involuntarily sitting next tw...
                              ...                        
7511    two years back someone invited tutor granddaug...
7512     taken responsibility something prepared howev...
7513     home heard loud sound spitting outside door t...
7514        homework teacher asked us scolded immediately
7515     shouted younger brother always afraid called ...
Name: dokumen, Length: 7516, dtype: object

In [22]:
df.shape

(7516, 2)

Lemmatization(harusnya setmmingkarna lematization = tokenize

In [23]:
from nltk.stem.snowball import SnowballStemmer

# Use English stemmer.
stemmer = SnowballStemmer("english")

In [24]:
sentences = [word_tokenize(x) for x in df.dokumen] 
# df = pd.DataFrame(df.apply(list,axis=1), columns=['dokumen'])

In [25]:
df.dokumen=sentences

In [26]:
# temp = df.dokumen.apply(lambda x: [stemmer.stem(y) for y in x])
df['dokumen'].apply(lambda x: [stemmer.stem(y) for y in x])

0       [day, feel, close, partner, friend, feel, peac...
1       [everi, time, imagin, someon, love, could, con...
2               [obvious, unjust, treat, possibl, elucid]
3       [think, short, time, live, relat, period, life...
4       [gather, found, involuntarili, sit, next, two,...
                              ...                        
7511    [two, year, back, someon, invit, tutor, grandd...
7512    [taken, respons, someth, prepar, howev, fail, ...
7513    [home, heard, loud, sound, spit, outsid, door,...
7514          [homework, teacher, ask, us, scold, immedi]
7515    [shout, younger, brother, alway, afraid, call,...
Name: dokumen, Length: 7516, dtype: object

In [27]:
df.head()

,emosi,dokumen
0,joy,"[days, feel, close, partner, friends, feel, pe..."
1,fear,"[every, time, imagine, someone, love, could, c..."
2,anger,"[obviously, unjustly, treated, possibility, el..."
3,sadness,"[think, short, time, live, relate, periods, li..."
4,disgust,"[gathering, found, involuntarily, sitting, nex..."


save df

In [ ]:
# df.to_csv(r'dataisear(preptambah).csv', index = False)

word2ec

In [28]:
from gensim.models import Word2Vec

In [29]:
len(df)

7516

In [30]:
w2v = Word2Vec(df.dokumen, window=2, min_count=1, workers=4, sg=1)

In [31]:
cek=w2v.wv
words=w2v.wv.vocab
# words
cek.vector_size

100

In [32]:
cek.vectors.shape

(9192, 100)

In [33]:
len(cek.vocab)

9192

In [34]:
cek["moment"]

array([ 0.3994058 , -0.04242042,  0.05068674,  0.3625266 , -0.10423772,
       -0.2813828 ,  0.23669934,  0.07620604,  0.03999291, -0.15281263,
        0.17039475, -0.2879122 ,  0.10627276,  0.0155327 , -0.11135358,
        0.14830956,  0.21832906,  0.14103836,  0.22177249,  0.08520136,
        0.23206519, -0.17316632, -0.10622631, -0.05822467, -0.05140306,
        0.3589519 ,  0.02611077,  0.32063672, -0.02540326, -0.14300208,
       -0.04141286, -0.27527273,  0.2817735 ,  0.04373953,  0.26488745,
       -0.00447474, -0.254749  ,  0.08547059, -0.012865  , -0.04945774,
       -0.14605953, -0.3625501 ,  0.02613487, -0.2862426 , -0.06269   ,
        0.00150189, -0.4579253 ,  0.10123256,  0.02525557, -0.39759168,
       -0.26668364,  0.0507686 , -0.16608126,  0.30251214, -0.31194386,
        0.0888268 , -0.12368622, -0.00207066, -0.04267465,  0.05682117,
       -0.20841591, -0.07763851, -0.08879895, -0.14756341,  0.33256343,
        0.08439235,  0.24784566,  0.00522536, -0.22280185, -0.07

ekstraksi fitur

In [38]:
import array
ekstrak=[]
temp=array.array('i',(0,)*100)
for senten in df.dokumen:
  for word in senten:
    temp=np.add(temp,cek[word])
  temp=temp/len(senten)
  ekstrak.append(temp)
has_vec=ekstrak

In [40]:
X=has_vec
y=df.emosi
print(np.shape(X))
print(np.shape(y))
# print(X)
# print(y)

(7516, 100)
(7516,)


klasifikasi svm

In [41]:
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [46]:
clf = svm.SVC(kernel='linear', decision_function_shape='ovr')# clf.kernel
scores = cross_val_score(clf, X, y, cv=10)
scores.mean()

0.22072661982604752

In [43]:
clf = svm.SVC(kernel='rbf', decision_function_shape='ovr')# clf.kernel
scores = cross_val_score(clf, X, y, cv=10)
scores.mean()

0.20183638127886222

In [44]:
clf = svm.SVC(kernel='poly', decision_function_shape='ovr')# clf.kernel
scores = cross_val_score(clf, X, y, cv=10)
scores.mean()

0.19385411649715273

In [45]:
clf = svm.SVC(kernel='sigmoid', decision_function_shape='ovr')# clf.kernel
scores = cross_val_score(clf, X, y, cv=10)
scores.mean()

0.12387207128084539

conf matrix cara 1

In [ ]:
y_pred = cross_val_predict(clf, X, y, cv=10)

In [ ]:
conf_mat = confusion_matrix(y, y_pred)
conf_mat

In [ ]:
# !pip install jcopml
# from jcopml.plot import plot_confusion_matrix

In [ ]:
accuracy_score(y, y_pred)

conf matrix cara 2

In [ ]:
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
clf.fit(X, y)
plot_confusion_matrix(clf, X, y)
plt.show()